In [10]:
!pip install python-docx
import requests
import pandas as pd
from docx import Document
from google.colab import files

# Your read-only token (starts with 'api:')
api_token = 'api:NTc4NGZjMzFkMjNhODNmMmYxNzExNDhlNzFiZmY2OWMyYTFkYTFhYzlmZTBmZTIzZWRmOTM2OGQ5YzVhMmNkMQ'

# Organization ID (replace with your actual organization ID)
organization_id = '638ff0f4a2101f386901bf22'

# API endpoint to get circles in the organization
api_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/circles?page=1&count=50'

# Headers for the GET request
headers = {
    'Authorization': f'Bearer {api_token}',
    'Accept': 'application/json'
}

# Function to get role details and fetch lead's name
def get_lead_member_name(role_id):
    if role_id == "N/A" or not role_id:
        return "No Lead Assigned"

    # API URL to get role details
    role_api_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/roles/{role_id}'
    role_response = requests.get(role_api_url, headers=headers)

    if role_response.status_code == 200:
        role_data = role_response.json()

        # Check if role has assigned members
        if 'linked' in role_data and 'members' in role_data['linked'] and role_data['linked']['members']:
            # Get the first member's display name
            member_name = role_data['linked']['members'][0].get('displayName', 'No Member Name Found')
            return member_name
        else:
            return "No Lead Assigned"
    else:
        return "Unknown Lead"

# Function to get parent circle name using circle ID
def get_circle_name(circle_id):
    if circle_id == "N/A" or not circle_id:
        return "No Parent Circle"

    # API URL to get circle details
    circle_api_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/circles/{circle_id}'
    circle_response = requests.get(circle_api_url, headers=headers)

    if circle_response.status_code == 200:
        circle_data = circle_response.json()
        return circle_data.get('data', {}).get('name', 'Unknown Circle')
    else:
        return "Unknown Circle"

# Send the GET request to fetch circles
response = requests.get(api_url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    print("Data successfully retrieved!")
    data = response.json()

    # Let's extract relevant fields from the circles and store them in a DataFrame
    circles_data = []

    for circle in data['data']:
        circle_info = {
            'Circle ID': circle['id'],
            'Name': circle['name'],
            'Lead ID': circle.get('lead', 'N/A'),  # Get the lead role ID
            'Parent Circle ID': circle.get('parentCircle', 'N/A'),
            'Roles Count': circle['rolesCount'],
            'Members Count': circle['membersCount'],
            'Created At': circle['createdAt'],
            'Updated At': circle['updatedAt']
        }

        # Fetch the lead member name using the role ID
        circle_info['Lead Name'] = get_lead_member_name(circle_info['Lead ID'])

        # Fetch the parent circle name using the parent circle ID
        circle_info['Parent Circle Name'] = get_circle_name(circle_info['Parent Circle ID'])

        circles_data.append(circle_info)

    # Create a pandas DataFrame for better visualization
    df_CircleData = pd.DataFrame(circles_data)

    # Print the DataFrame before downloading
    print(df_CircleData)

    # Convert the DataFrame to a .docx file
    doc = Document()
    doc.add_heading('Circle Data', 0)

    # Add table with columns
    table = doc.add_table(rows=1, cols=len(df_CircleData.columns))
    hdr_cells = table.rows[0].cells
    for i, column_name in enumerate(df_CircleData.columns):
        hdr_cells[i].text = column_name

    # Add data rows
    for index, row in df_CircleData.iterrows():
        row_cells = table.add_row().cells
        for i, value in enumerate(row):
            row_cells[i].text = str(value)

    # Save the document
    doc_filename = 'Circle_Data.docx'
    doc.save(doc_filename)

    # Download the document in Google Colab
    files.download(doc_filename)

else:
    print(f"Failed to access the API. Status code: {response.status_code}")
    print("Error message:", response.text)


Data successfully retrieved!
                   Circle ID                                         Name  \
0   638ff0f6a2101f386901c0ea                             Asset Management   
1   638ff0f6a2101f386901bf4d                                    Batteries   
2   66e020263cffacf97d09a48e              Battery Brunsbüttel Development   
3   638ff0f6a2101f386901bf6f                          Battery Engineering   
4   656f019a26b427d0770ee59c                                Battery@Solar   
5   638ff0f6a2101f386901bfb3                          Business Excellence   
6   656341d037d713f77b0ff843                          Contract Management   
7   638ff0f6a2101f386901c154       Controlling & Financial Administration   
8   638ff0f6a2101f386901c014                                     Delivery   
9   638ff0f6a2101f386901c034                        Delivery Construction   
10  659e5cc6c375d6022b0582dd                   Digital project management   
11  638ff0f6a2101f386901c052                   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# Calculate total number of roles and unique members
total_roles =  df_CircleData['Roles Count'].sum()


print(f"Total Roles: {total_roles}")


Total Roles: 316


This code get all the circles, then get all roles within the circles and then get role infos of all roles. In this way, Assigned Members and their Accountabilities are refreshed


In [11]:
import requests
import pandas as pd
from docx import Document
from google.colab import files

# Initialize counters and dataframe
role_counter = 0
unique_members = set()  # Use a set to store unique member IDs
roles_data_list = []    # List to store data for DataFrame

# Function to get all circles from the organization
def get_all_circles(organization_id, api_token):
    api_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/circles'
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Accept': 'application/json'
    }

    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve circles: {response.status_code}")
        return None

# Function to get roles in a specific circle
def get_roles_in_circle(organization_id, circle_id, api_token):
    api_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/roles-custom-fields?circle={circle_id}'
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Accept': 'application/json'
    }

    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve roles for circle {circle_id}: {response.status_code}")
        return None

# Function to get role details
def get_role_details(organization_id, role_id, api_token):
    api_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/roles/{role_id}'
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Accept': 'application/json'
    }

    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve role data for {role_id}: {response.status_code}")
        return None

# Function to display role member names, accountabilities, and purpose
def display_role_info_and_count(role_data, unique_members, circle_name, role_name):
    global role_counter

    # Check if the role has assigned members
    if 'linked' in role_data and 'members' in role_data['linked']:
        # Fetch all assigned member names (in case there are multiple members)
        member_names = [member.get('displayName', 'No Member Assigned') for member in role_data['linked']['members']]
        # Add member IDs to the unique members set
        member_ids = [member.get('id') for member in role_data['linked']['members']]
        unique_members.update(member_ids)
    else:
        member_names = ['No Member Assigned']

    # Fetch accountability descriptions from the 'linked' section
    accountability_descriptions = [
        accountability.get('description', 'No description available')
        for accountability in role_data['linked'].get('accountabilities', [])
    ]

    # Fetch the purpose from role_data
    purpose = role_data['data'].get('purpose', 'No purpose provided')

    # Add data to the roles_data_list for DataFrame
    roles_data_list.append({
        'Role Name': role_name,
        'Circle Name': circle_name,
        'Assigned Members': ', '.join(member_names),
        'Accountabilities': ', '.join(accountability_descriptions) if accountability_descriptions else 'No Accountabilities',
        'Purpose': purpose
    })

    # Print members, accountabilities, and purpose
    print(f"Assigned Members: {', '.join(member_names)}")
    print(f"Accountabilities: {', '.join(accountability_descriptions) if accountability_descriptions else 'No Accountabilities'}")
    print(f"Purpose: {purpose}")
    print("-" * 40)

    # Increment role counter
    role_counter += 1

# Main function to retrieve and process all circles and roles
def process_circles_and_roles_count(organization_id, api_token):
    # Step 1: Get all circles in the organization
    circles_data = get_all_circles(organization_id, api_token)

    if circles_data:
        # Step 2: For each circle, get roles and process them
        all_circles = circles_data.get('data', [])

        for circle in all_circles:
            circle_id = circle.get('id')
            circle_name = circle.get('name', 'Unknown Circle')

            print(f"Processing Circle: {circle_name}")

            # Step 3: Get all roles in the circle
            roles_data = get_roles_in_circle(organization_id, circle_id, api_token)

            if roles_data:
                all_roles = roles_data.get('data', [])

                for role in all_roles:
                    role_id = role.get('id')
                    role_name = role.get('name', 'Unknown Role')

                    print(f"Processing Role: {role_name}")
                    print(f"Processing Circle: {circle_name}")

                    # Step 4: For each role, get role details
                    role_data = get_role_details(organization_id, role_id, api_token)

                    if role_data:
                        # Step 5: Display member names, accountabilities, and purpose while counting roles
                        display_role_info_and_count(role_data, unique_members, circle_name, role_name)

# Example usage
organization_id = "638ff0f4a2101f386901bf22"  # Replace with your organization ID
api_token = "api:NTc4NGZjMzFkMjNhODNmMmYxNzExNDhlNzFiZmY2OWMyYTFkYTFhYzlmZTBmZTIzZWRmOTM2OGQ5YzVhMmNkMQ"  # Replace with your API token

process_circles_and_roles_count(organization_id, api_token)

# Create a DataFrame from the collected roles data
df_RolesData = pd.DataFrame(roles_data_list)

# Display the DataFrame
print(df_RolesData)

# Save the DataFrame to a .docx file
doc = Document()
doc.add_heading('Roles Data', 0)

# Add table with columns
table = doc.add_table(rows=1, cols=len(df_RolesData.columns))
hdr_cells = table.rows[0].cells
for i, column_name in enumerate(df_RolesData.columns):
    hdr_cells[i].text = column_name

# Add data rows
for index, row in df_RolesData.iterrows():
    row_cells = table.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

# Save the document
doc_filename = 'Roles_Data.docx'
doc.save(doc_filename)

# Download the document in Google Colab
files.download(doc_filename)


Processing Circle: Asset Management
Processing Role: Asset Manager
Processing Circle: Asset Management
Assigned Members: Jaco Hovius, Ahmed Altarawneh, Siddharth Dutta, Srinivas Krishna, Maaike Hermse
Accountabilities: Planning, Organizing, controlling the quality of all activities (including maintenance) on the Asset after TG5., Ensuring all work on the asset is performed safely, Keeping relevant Asset related documentation (including HSE documentation, technical documentation, maintenance & inspection plans and documentation for regulatory requirements) up to date, Performing Safety walks on the assets and creating and following up on Intelex observations on the assets., Ensuring insurance requirements are met and the assets and services are insured up to the right level., Ensuring TG5 budgets & production are kept, both in costs & revenues., Preparing the yearly Business Plan for the Assets together with Financial Control, Preparing & implementing improvement proposals to further op

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# Print the total number of roles and unique members
print("=" * 60)
print(f"Total Roles Processed: {role_counter}")
print(f"Total Unique Members: {len(unique_members)}")
print("=" * 60)


Total Roles Processed: 316
Total Unique Members: 128


In [16]:
import requests
import pandas as pd
from docx import Document
from google.colab import files

# Initialize list to store relationship data for DataFrame
relations_data_list = []

# Initialize a counter for the number of connected roles
role_connection_counter = 0

# Function to get role relations
def get_role_relations(organization_id, api_token):
    relations_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/roles/relations'
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Accept': 'application/json'
    }
    response = requests.get(relations_url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve role relations: {response.status_code}")
        return None

# Function to get role info
def get_role_info(organization_id, role_id, api_token):
    role_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/roles/{role_id}'
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Accept': 'application/json'
    }
    response = requests.get(role_url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve role info for role_id {role_id}: {response.status_code}")
        return None

# Function to get circle name using circle ID
def get_circle_name(organization_id, circle_id, api_token):
    circle_url = f'https://app.holaspirit.com/api/organizations/{organization_id}/circles/{circle_id}'
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Accept': 'application/json'
    }
    response = requests.get(circle_url, headers=headers)

    if response.status_code == 200:
        circle_data = response.json()
        return circle_data['data'].get('name', 'Unknown Circle')
    else:
        print(f"Failed to retrieve circle info for circle_id {circle_id}: {response.status_code}")
        return 'Unknown Circle'

# Function to display and store role relationships with role names and circle names
def display_role_relationships_with_names_and_circles(organization_id, relations_data, api_token):
    global relations_data_list
    global role_connection_counter

    for relation in relations_data['data']:
        origin_role_id = relation['origin']
        target_role_id = relation['target']

        # Fetch role names and circle info using the role info API
        origin_role_data = get_role_info(organization_id, origin_role_id, api_token)
        target_role_data = get_role_info(organization_id, target_role_id, api_token)

        # Get role names and circle IDs
        origin_role_name = origin_role_data['data'].get('name', 'Unknown Role') if origin_role_data else 'Unknown Role'
        target_role_name = target_role_data['data'].get('name', 'Unknown Role') if target_role_data else 'Unknown Role'

        # Fetch the circle name using the circle ID
        origin_circle_id = origin_role_data['data'].get('circle', None) if origin_role_data else None
        target_circle_id = target_role_data['data'].get('circle', None) if target_role_data else None

        origin_circle_name = get_circle_name(organization_id, origin_circle_id, api_token) if origin_circle_id else 'No Circle'
        target_circle_name = get_circle_name(organization_id, target_circle_id, api_token) if target_circle_id else 'No Circle'

        # Display the relation with role names and circle names
        print(f"Role {origin_role_name} (Circle: {origin_circle_name}) is related to Role {target_role_name} (Circle: {target_circle_name})")

        # Append data to the list for DataFrame
        relations_data_list.append({
            'Origin Role': origin_role_name,
            'Origin Circle': origin_circle_name,
            'Target Role': target_role_name,
            'Target Circle': target_circle_name
        })

        # Increment the counter for role connections
        role_connection_counter += 1

# Example usage
organization_id = "638ff0f4a2101f386901bf22"  # Replace with your organization ID
api_token = "api:NTc4NGZjMzFkMjNhODNmMmYxNzExNDhlNzFiZmY2OWMyYTFkYTFhYzlmZTBmZTIzZWRmOTM2OGQ5YzVhMmNkMQ"  # Replace with your API token

# Step 1: Fetch role relations
role_relations_data = get_role_relations(organization_id, api_token)

# Step 2: Display role relationships with role names and circles, and store them
if role_relations_data:
    display_role_relationships_with_names_and_circles(organization_id, role_relations_data, api_token)
else:
    print("No role relations found.")

# Create a DataFrame from the collected relationships data
df_RelationsData = pd.DataFrame(relations_data_list)

# Display the DataFrame
print(df_RelationsData)

# Save the DataFrame to a .docx file
doc = Document()
doc.add_heading('Role Relations Data', 0)

# Add table with columns
table = doc.add_table(rows=1, cols=len(df_RelationsData.columns))
hdr_cells = table.rows[0].cells
for i, column_name in enumerate(df_RelationsData.columns):
    hdr_cells[i].text = column_name

# Add data rows
for index, row in df_RelationsData.iterrows():
    row_cells = table.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

# Save the document
doc_filename = 'Role_Relations_Data.docx'
doc.save(doc_filename)

# Download the document in Google Colab
files.download(doc_filename)


Role Cross Link to @Operations (Circle: No Circle) is related to Role Batteries (Circle: Batteries)
Role Cross Link to @Operations (Circle: No Circle) is related to Role Asset Management (Circle: Asset Management)
Role Technology Interface (Circle: No Circle) is related to Role Battery Engineering (Circle: Battery Engineering)
Role Technology Interface (Circle: No Circle) is related to Role Battery-Engineering Book Keeper (Circle: No Circle)
Role Circle Lead (Circle: No Circle) is related to Role Batteries (Circle: Batteries)
Role Delivery (Circle: Delivery) is related to Role Solar Development (Circle: Solar Development)
Role Engineering (Circle: Engineering) is related to Role Solar Development (Circle: Solar Development)
Role Engineering (Circle: Engineering) is related to Role Delivery Construction (Circle: Delivery Construction)
Role Engineering (Circle: Engineering) is related to Role Asset Management (Circle: Asset Management)
Role Asset Management (Circle: Asset Management) is 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Print the number of role connections
print(f"Total number of role connections: {role_connection_counter}")

# Optionally, display the DataFrame
df_RelationsData


Total number of role connections: 87


,Origin Role,Origin Circle,Target Role,Target Circle
0,Cross Link to @Operations,No Circle,Batteries,Batteries
1,Cross Link to @Operations,No Circle,Asset Management,Asset Management
2,Technology Interface,No Circle,Battery Engineering,Battery Engineering
3,Technology Interface,No Circle,Battery-Engineering Book Keeper,No Circle
4,Circle Lead,No Circle,Batteries,Batteries
...,...,...,...,...
82,M&A transaction manager,No Circle,M&A,No Circle
83,Project Administration Support,No Circle,Project Manager,No Circle
84,Project Administration Support,No Circle,BESS Engineering Manager,No Circle
85,Legal Entity Coordinator,No Circle,Portfolio Manager,No Circle


In [17]:
df_CircleData
#df_RolesData
#df_RelationsData

,Circle ID,Name,Lead ID,Parent Circle ID,Roles Count,Members Count,Created At,Updated At,Lead Name,Parent Circle Name
0,638ff0f6a2101f386901c0ea,Asset Management,638ff0f6a2101f386901c0ec,638ff0f6a2101f386901c014,17,6,2022-12-07T01:48:38+00:00,2024-08-26T15:38:02+00:00,Jaco Hovius,Delivery
1,638ff0f6a2101f386901bf4d,Batteries,638ff0f6a2101f386901bf4f,638ff0f4a2101f386901bf2a,26,27,2022-12-07T01:48:38+00:00,2024-09-13T09:30:27+00:00,Sebastian Gerhard,Solar and Batteries
2,66e020263cffacf97d09a48e,Battery Brunsbüttel Development,66e020263cffacf97d09a49a,638ff0f6a2101f386901bf4d,14,2,2024-09-10T10:32:06+00:00,2024-09-17T07:54:05+00:00,No Lead Assigned,Batteries
3,638ff0f6a2101f386901bf6f,Battery Engineering,638ff0f6a2101f386901bf71,638ff0f6a2101f386901bf4d,19,18,2022-12-07T01:48:38+00:00,2024-09-04T09:56:17+00:00,Philipp Degenhardt,Batteries
4,656f019a26b427d0770ee59c,Battery@Solar,656f019b26b427d0770ee5b0,638ff0f6a2101f386901bf4d,15,12,2023-12-05T10:55:23+00:00,2024-09-13T06:59:34+00:00,Paulina Asbeck,Batteries
5,638ff0f6a2101f386901bfb3,Business Excellence,638ff0f6a2101f386901bfb5,638ff0f4a2101f386901bf2a,23,10,2022-12-07T01:48:38+00:00,2024-08-29T10:08:52+00:00,Nicola Kleihues,Solar and Batteries
6,656341d037d713f77b0ff843,Contract Management,656341d137d713f77b0ff84a,638ff0f6a2101f386901c014,7,4,2023-11-26T13:02:09+00:00,2024-08-26T15:38:02+00:00,Diederik Hazenberg,Delivery
7,638ff0f6a2101f386901c154,Controlling & Financial Administration,638ff0f6a2101f386901c156,638ff0f4a2101f386901bf2a,11,8,2022-12-07T01:48:38+00:00,2024-08-26T15:38:02+00:00,Philipp Ebeling,Solar and Batteries
8,638ff0f6a2101f386901c014,Delivery,638ff0f6a2101f386901c016,638ff0f4a2101f386901bf2a,14,9,2022-12-07T01:48:38+00:00,2024-09-04T08:40:32+00:00,Bas van Kesteren,Solar and Batteries
9,638ff0f6a2101f386901c034,Delivery Construction,638ff0f6a2101f386901c036,638ff0f6a2101f386901c014,11,14,2022-12-07T01:48:38+00:00,2024-09-04T08:39:40+00:00,No Lead Assigned,Delivery
